<h2>Space Invaders Agent</h2>

In [3]:
import gym

<h3>Create environment:</h3>

In [4]:
env = gym.make('SpaceInvaders-v0')

In [5]:
env.action_space

Discrete(6)

In [6]:
episodes = 10

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        next_sate, reward, done, info = env.step(env.action_space.sample())
        score += reward
    
    print('Episode: {}\nScore: {}'.format(episode, score))

env.close()

d:\anaconda\lib\site-packages\gym\envs\atari\environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(
d:\anaconda\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Episode: 1
Score: 115.0
Episode: 2
Score: 130.0
Episode: 3
Score: 160.0
Episode: 4
Score: 160.0
Episode: 5
Score: 125.0
Episode: 6
Score: 210.0
Episode: 7
Score: 55.0
Episode: 8
Score: 100.0
Episode: 9
Score: 55.0


<h3>Build the Neural Network:</h3>

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
#Dense -> fully connected neural networks layers
#Flatten -> flat the results of the previous layers creating a 1D array
#Conv2D -> convolutional layer is necessary for the computer to understand images 
from tensorflow.keras.layers import Dense, Flatten, Conv2D
#Adam -> is what allow us to find the optimal weights of our neural network (= optimal agent that can interact with the environment)
from tensorflow.keras.optimizers import Adam

In [8]:
env.observation_space.shape

(210, 160, 3)

In [9]:
# th parameters "height, width, channels" are the pixels of our screen
def build_model(height, width, channels, actions):
    model = Sequential()
    # 32 output units
    # (8,8), strides=(4, 4) -> the variables used to condense the image so our model can understand the image in a more computationaly friendly way
    # activation='relu' -> relu allows us to have some unlinearity inside of our model so the model can learn more comlexe pattern
    # input_shape -> will take an input of our screen and then we will output an action based on the screen
    model.add(Conv2D(32, (8,8), strides=(4, 4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2, 2), activation='relu'))
    #flatten our output into one dimensional array
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))

    return model

In [10]:
env.action_space.n

6

In [11]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [13]:
model = build_model(height, width, channels, actions)